# ReACT Agent

For many, a Large Language Model (LLM) Agent is synonymous with the ReACT architecture. This simple yet powerful architecture is present in most mainstream Agentic frameworks. In this notebook, I aim to explain how ReACT works under the hood and demonstrate that HybridAGI, as a general-purpose, programmable graph-based Agentic framework, can indeed build a ReACT agent.

However, we do not advocate for the ReACT architecture as the ultimate solution, especially for complex tasks, due to its lack of control and efficiency. Nevertheless, if you're new to Graph-based Prompt Programming, a ReACT agent can serve as an excellent starting point. With HybridAGI, you can iteratively enhance this initial agent and deviate from the ReACT architecture by adding more actions and decisions, all within the same framework. This flexibility is one of the key strengths of HybridAGI.

In its essence, a ReACT agent is composed by two elements:

- A decision for choosing the tool to use
- A loop to iteratively adapt to the user input

In [1]:
# Let's start implementing it!

import hybridagi.core.graph_program as gp

main = gp.GraphProgram(
    name = "main",
    description = "The main program",
)

main.add(gp.Decision(
    id = "tool_choice",
    purpose = "Choose the next tool to use",
    question = \
"""Which tool to use for the next step?
Use the context to help you choose.
To give the final answer just select finish""",
))

main.add(gp.Action(
    id = "ask_user",
    purpose = "Ask the user",
    tool = "AskUser",
    prompt = "Ask a question to the user",
))

main.add(gp.Action(
    id = "finish",
    purpose = "End the conversation and give the final answer",
    tool = "Speak",
    prompt = "Please give the final answer, if you don't know just say that you don't know",
))

main.connect("start", "tool_choice")
main.connect("tool_choice", "ask_user", label="Ask User")
main.connect("tool_choice", "finish", label="Finish")
main.connect("ask_user", "tool_choice")
main.connect("finish", "end")

main.build()

print(main)



/home/yoan/.cache/pypoetry/virtualenvs/hybridagi-B1GoJrSC-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


// @desc: The main program
CREATE
// Nodes declaration
(start:Control {id: "start"}),
(end:Control {id: "end"}),
(tool_choice:Decision {
  id: "tool_choice",
  purpose: "Choose the next tool to use",
  question: "Which tool to use for the next step?\nUse the context to help you choose.\nTo give the final answer just select finish"
}),
(ask_user:Action {
  id: "ask_user",
  purpose: "Ask the user",
  tool: "AskUser",
  prompt: "Ask a question to the user"
}),
(finish:Action {
  id: "finish",
  purpose: "End the conversation and give the final answer",
  tool: "Speak",
  prompt: "Please give the final answer, if you don't know just say that you don't know"
}),
// Structure declaration
(start)-[:NEXT]->(tool_choice),
(tool_choice)-[:ASK_USER]->(ask_user),
(tool_choice)-[:FINISH]->(finish),
(ask_user)-[:NEXT]->(tool_choice),
(finish)-[:NEXT]->(end)


In [2]:
# Now we setup our agent

import dspy
from hybridagi.core.datatypes import AgentState, Query
from hybridagi.modules.agents import GraphInterpreterAgent
from hybridagi.memory.integration.local import LocalProgramMemory
from hybridagi.modules.agents.tools import (
    SpeakTool,
    AskUserTool,
)

program_memory = LocalProgramMemory(index_name="react_agent")

# Never forget to load your main program into memory!

program_memory.update(main)

# Setup the tools

agent_state = AgentState()

tools = [
    SpeakTool(
        agent_state = agent_state,
    ),
    AskUserTool(
        agent_state = agent_state,
    ),
]

agent = GraphInterpreterAgent(
    program_memory = program_memory,
    agent_state = agent_state,
    tools = tools,
)

# Don't forget to setup you LLM

lm = dspy.OllamaLocal(model='mistral', max_tokens=1024, stop=["\n\n\n", "\n---"])
dspy.configure(lm=lm)

result = agent(Query(query="Please teach me how to how to bake a chocolate, vanilla or strawberry cake"))

print(result.final_answer)

--- Step 0 ---
Call Program: main
Program Purpose: Please teach me how to how to bake a chocolate, vanilla or strawberry cake
--- Step 1 ---
Decision Purpose: Choose the next tool to use
Decision Question: Which tool to use for the next step?
Use the context to help you choose.
To give the final answer just select finish
Decision: ASK_USER
--- Step 2 ---
Action Purpose: Ask the user
Action: {
  "question": "What flavor of cake would you like to bake, chocolate, vanilla or strawberry?",
  "answer": "Chocolate cake, please."
}
--- Step 3 ---
Decision Purpose: Choose the next tool to use
Decision Question: Which tool to use for the next step?
Use the context to help you choose.
To give the final answer just select finish
Decision: ASK_USER
--- Step 4 ---
Action Purpose: Ask the user
Action: {
  "question": "What flavor of cake would you like to bake, chocolate, vanilla or strawberry?",
  "answer": "Chocolate cake, as previously stated."
}
--- Step 5 ---
Decision Purpose: Choose the next t

So next time that you see a ReACT agent, you should have this structure in mind. And you should know why they are not the future of Agentic systems and why you should forget about any framework that emphasize their use.